# Quick MNIST

*by Jeremy Howard, fast.ai.*

In this notebook we'll see how easy it is to use the simple functions we created along with `torch.nn` and friends to train MNIST. We're going to create the same convolution net that we created at the end of the previous notebook, but this time we'll use the functions we've already written, and we'll skip the explanatory text so you can see just the final code.

You can use this same notebook to train other neural nets on other datasets with minimal changes.

In [ ]:
import pickle, gzip
from pathlib import Path
from mnist_sample import *

In [ ]:
PATH = Path('data')/'mnist'

with gzip.open(PATH/'mnist.pkl.gz', 'rb') as f:
    ((train_x, train_y), (valid_x, valid_y), _) = pickle.load(f, encoding='latin-1')

In [ ]:
bs=64
lr=0.1
epochs=2

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
def preprocess(x,y): return x.view(-1,1,28,28).to(dev),y.to(dev)

def get_dataloader(x,y,bs,shuffle):
    ds = TensorDataset(*map(tensor, (x,y)))
    dl = DataLoader(ds, batch_size=bs, shuffle=shuffle)
    return WrappedDataLoader(dl, preprocess)

In [ ]:
train_dl = get_dataloader(train_x, train_y, bs,   shuffle=False)
valid_dl = get_dataloader(valid_x, valid_y, bs*2, shuffle=True )

In [ ]:
model = nn.Sequential(
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.view(x.size(0),-1))
).to(dev)

opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [ ]:
fit(epochs, model, F.cross_entropy, opt, train_dl, valid_dl)

0 0.34885788760185243
1 0.2653279606819153
